In [59]:
from kaldi.feat.mfcc import Mfcc, MfccOptions
from kaldi.matrix import SubVector, SubMatrix
from kaldi.util.options import ParseOptions
from kaldi.util.table import SequentialWaveReader
from kaldi.util.table import MatrixWriter
from numpy import mean
from sklearn.preprocessing import scale, MinMaxScaler, OneHotEncoder
import numpy as np
import os
import keras
from keras.layers import Input, Dense, Conv1D, Conv2D, Reshape, MaxPooling1D, Flatten, UpSampling1D
from keras.layers import TimeDistributed, RepeatVector
from keras.models import Model
import tensorflow as tf
scaler = MinMaxScaler
from sklearn.model_selection import train_test_split
from time import time

In [2]:
start_time = time()
current_time = start_time
def log(message):
    global current_time
    t = time()
    print(f"{message}, {int(t - current_time)}s, total {int(t - start_time)}s")
    current_time = t
def start():
    global start_time
    global current_time
    start_time = time()
    current_time = start_time

In [53]:
# np.save("../data/mfcc.npy", X_mfcc)
# np.save("../data/mfcc_noised.npy", X_mfcc_noised)
X_mfcc = np.load("../data/mfcc.npy")
X_mfcc_noised = np.load("../data/mfcc_noised.npy")

In [65]:
test_size = 0.1
X_mfcc_train, X_mfcc_test, X_mfcc_noised_train, X_mfcc_noised_test, y_train, y_test =\
    train_test_split(X_mfcc, X_mfcc_noised, y, test_size=test_size, stratify=y)


In [3]:
# np.save("../data/mfcc_train.npy", X_mfcc_train)
# np.save("../data/mfcc_test.npy", X_mfcc_test)
# np.save("../data/mfcc_noised_train.npy", X_mfcc_noised_train)
# np.save("../data/mfcc_noised_test.npy", X_mfcc_noised_test)
X_mfcc_noised_train = np.load("../data/mfcc_noised_train.npy")
X_mfcc_noised_test = np.load("../data/mfcc_noised_test.npy")

In [4]:
# np.save("../data/y_train.npy", y_train)
# np.save("../data/y_test.npy", y_test)
y_train = np.load("../data/y_train.npy")
y_test = np.load("../data/y_test.npy")

In [9]:
# def Conv1DTranspose(input_tensor, filters, kernel_size, strides=2, padding='same', activation='relu'):
#     x = keras.layers.Lambda(lambda x: keras.backend.expand_dims(x, axis=2))(input_tensor)
#     x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1),
#                         padding=padding, activation=activation)(x)
#     x = keras.layers.Lambda(lambda x: keras.backend.squeeze(x, axis=2))(x)
#     return x
# def sampling(args):
#     z_mean, z_log_sigma = args
#     epsilon = keras.backend.random_normal(shape=(batch_size, latent_dim),
#                               mean=0., stddev=1.)
#     return z_mean + keras.backend.exp(z_log_sigma) * epsilon

In [70]:
def train_generator(X, Y):
    for i in range(X.shape[0]):
        x = MinMaxScaler().fit_transform(X[i])
        x = x.reshape(1, *x.shape)
        y = MinMaxScaler().fit_transform(Y[i])
        y = y.reshape(1, *y.shape)
        yield (x, y)
def test_generator(X, k):
    x = MinMaxScaler().fit_transform(X[k])
    x = x.reshape(1, *x.shape)
    yield x

In [72]:
# test_k = 32735
# pred = rnn_ae.predict_generator(test_generator(X_mfcc_noised),
# #                      epochs=10,
#                      verbose=1,
#                      steps=1)
# true = MinMaxScaler().fit_transform(X_mfcc[test_k])
# noisy = MinMaxScaler().fit_transform(X_mfcc_noised[test_k])
# print(((pred - true)**2).mean())
# print(((noisy - true)**2).mean()) 
# # ((pred.reshape(*pred.shape[1:]) - X_mfcc[0])**2).mean()
# # r1 = keras.losses.mse(pred.reshape(*pred.shape[1:]),
# #                                  tf.convert_to_tensor(X_mfcc[0]))
# # r2 = keras.losses.mse(tf.convert_to_tensor(X_mfcc_noised[0]),
#                                  tf.convert_to_tensor(X_mfcc[0]))
def predict_ae(X):
    res = []
    for i in range(X.shape[0]):
        if i % 500 == 0:
            log(f"{i}/{X.shape[0]}")
        pred = rnn_ae.predict_generator(test_generator(X, i),
                                        verbose=0,
                                        steps=1)
        pred = pred.reshape(*pred.shape[1:])
        res.append(pred)
    return res


In [5]:
ae_features_noised_train = np.load("../data/ae_features_noised_train.npy")
ae_features_noised_test = np.load("../data/ae_features_noised_test.npy")

In [7]:
num_labels = len(set(y_train))
num_labels

109

In [131]:

mfcc_input = Input(shape=(None, 13), name='mfcc_input')
gru1 = keras.layers.GRU(64,
                        stateful=False,
                        return_sequences=False)(mfcc_input)
# aux_output = TimeDistributed(Dense(num_labels, activation='softmax', name='aux_output'))(gru1)
aux_output = Dense(num_labels, activation='softmax', name='aux_output')(gru1)

ae_input = Input(shape=(None, 12), name='ae_input')
gru2 = keras.layers.GRU(20,
                        stateful=False,
                        return_sequences=False)(ae_input)

x = keras.layers.concatenate([gru1, gru2])
# x = TimeDistributed(Dense(64, activation='relu'))(x)
# x = TimeDistributed(Dense(64, activation='relu'))(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# main_output = TimeDistributed(Dense(num_labels, activation='softmax', name='main_output'))(x)
main_output = Dense(num_labels, activation='softmax', name='main_output')(x)

model = Model(inputs=[mfcc_input, ae_input], outputs=[main_output, aux_output])

model.compile(optimizer='adam', loss='categorical_crossentropy')

In [164]:
def train_generator(X_main, X_aux, y):
    for i in range(X_main.shape[0]):
        mfcc = X_main[i]
        mfcc = mfcc.reshape(1, *mfcc.shape)
        ae = X_aux[i]
        ae = ae.reshape(1, *ae.shape)
#         local_y = np.array([y[i]] * mfcc.shape[1]).reshape(1, mfcc.shape[1], 1)
#         print(np.array([y[i].todense()] * mfcc.shape[1]).shape)
#         raise Exception
#         local_y = np.array([y[i].todense()] * mfcc.shape[1]).reshape(1, mfcc.shape[1], num_labels)
        local_y = y[i].todense()
#         print(local_y.shape)
        yield ({'mfcc_input': mfcc,
                'ae_input': ae},
               {'main_output': local_y, 'aux_output': local_y})
def test_generator(X1, X2, k):
    x1 = X1[k]
    x1 = x1.reshape(1, *x1.shape)
    x2 = X2[k]
    x2 = x2.reshape(1, *x2.shape)
    yield ({'mfcc_input': x1,
            'ae_input': x2})

In [165]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mfcc_input (InputLayer)         (None, None, 13)     0                                            
__________________________________________________________________________________________________
ae_input (InputLayer)           (None, None, 12)     0                                            
__________________________________________________________________________________________________
gru_20 (GRU)                    (None, 64)           14976       mfcc_input[0][0]                 
__________________________________________________________________________________________________
gru_21 (GRU)                    (None, 20)           1980        ae_input[0][0]                   
__________________________________________________________________________________________________
concatenat

In [166]:
print(len(X_mfcc_noised_train[0]), len(ae_features_noised_train[0]), y_train.shape)
y_sparse_train = OneHotEncoder(categories='auto').fit_transform(y_train.reshape(-1, 1))
y_sparse_test = OneHotEncoder(categories='auto').fit_transform(y_test.reshape(-1, 1))

365 365 (39817,)


In [188]:
saver = keras.callbacks.ModelCheckpoint("./models/classifyer.checkpoint", verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
model.fit_generator(train_generator(X_mfcc_noised_train[::-1], ae_features_noised_train[::-1], y_sparse_train[::-1]),
          epochs=100,
          verbose=1,
          shuffle=True,
#           validation_data=({'mfcc_input': mfcc_seq_val, 'fbank_input': fbank_seq_val},
#                            {'main_output': y_val_seq, 'aux_output': y_val_seq}),
#           class_weight={0 : class_weights[0], 1 : class_weights[1]},
#           initial_epoch=0,
          callbacks = [saver],
          steps_per_epoch=100,
          validation_steps=10)

Epoch 1/100
100/100 [==============================] - 67s 666ms/step - loss: 7.9924 - main_output_loss: 4.0623 - aux_output_loss: 3.9300

Epoch 00001: saving model to ./models/classifyer.checkpoint
Epoch 2/100
100/100 [==============================] - 73s 729ms/step - loss: 8.3397 - main_output_loss: 4.2271 - aux_output_loss: 4.1126

Epoch 00002: saving model to ./models/classifyer.checkpoint
Epoch 3/100
100/100 [==============================] - 74s 739ms/step - loss: 7.7978 - main_output_loss: 3.8996 - aux_output_loss: 3.8983

Epoch 00003: saving model to ./models/classifyer.checkpoint
Epoch 4/100
100/100 [==============================] - 68s 684ms/step - loss: 8.0886 - main_output_loss: 4.1287 - aux_output_loss: 3.9599

Epoch 00004: saving model to ./models/classifyer.checkpoint
Epoch 5/100
100/100 [==============================] - 68s 685ms/step - loss: 8.1165 - main_output_loss: 4.0908 - aux_output_loss: 4.0257

Epoch 00005: saving model to ./models/classifyer.checkpoint
Epoch

KeyboardInterrupt: 

In [173]:
def predict(X1, X2):
    res = []
    for i in range(X1.shape[0]):
        if i % 500 == 0:
            log(f"{i}/{X1.shape[0]}")
        pred = model.predict_generator(test_generator(X1, X2, i),
                                       verbose=0,
                                       steps=1)
#         pred = pred.reshape(*pred.shape[1:])
        res.append(pred)
    return res


In [212]:
k = 500
start()
pred = predict(X_mfcc_noised_test, ae_features_noised_test)
# pred = pred.reshape(*pred.shape[1:])
# # np.argmax(.reshape(), axis=0)
# res = np.argmax(pred) == y_test[:k]

0/4425, 0s, total 0s
500/4425, 105s, total 105s
1000/4425, 104s, total 209s
1500/4425, 99s, total 308s
2000/4425, 105s, total 413s
2500/4425, 102s, total 516s
3000/4425, 104s, total 620s
3500/4425, 103s, total 723s
4000/4425, 102s, total 825s


In [213]:
predicted_labels = np.array([np.argmax(x[1]) for x in pred])

In [214]:
res = (predicted_labels == y_test)

In [216]:
res.sum() / res.shape[0]

0.07209039548022599